In [1]:
! pip install pandas
! pip install openpyxl

In [2]:
import pandas as pd

In [3]:
class MatchRating:
    original_rating_name = "Original"
    avg_player_name = "AVG"
    max_iterations = 1000

    def __init__(self, data_sheet_file, rating_scale_base, rating_scale_divisor, rating_base, tie_in_weight):

        self.rating_scale_base = rating_scale_base
        self.rating_scale_divisor = rating_scale_divisor
        self.ranking_boost = rating_base
        self.tie_in_weight = tie_in_weight
        
        self.players: pd.DataFrame = pd.read_excel(data_sheet_file, sheet_name="Players", index_col="ShortName")
        self.matches: pd.DataFrame = pd.read_excel(data_sheet_file, sheet_name="Matches")

        # --- Add tie-in matches ---
        cols = self.matches.columns
        first_winner = cols[cols.str.startswith("Winner")].values[0]
        first_loser = cols[cols.str.startswith("Loser")].values[0]
        
        tie_in_rows = []
        for player in self.players.index:
            tie_in_rows.append({first_winner: player, first_loser: MatchRating.avg_player_name, "K": self.tie_in_weight})
            tie_in_rows.append({first_winner: MatchRating.avg_player_name, first_loser: player, "K": self.tie_in_weight})

        tie_in_matches = pd.DataFrame(tie_in_rows)
        self.matches = pd.concat([self.matches, tie_in_matches], ignore_index=True)

        self.__player_ratings: pd.DataFrame = pd.DataFrame(columns=["Rating"], index=self.players.index)
        self.__player_ratings["Rating"] = 0.0
        

    def calculate_ratings(self, step_sizes: list):

        for step_size in step_sizes:
            rankings = self.__calculate_ratings(step_size)

        excluded_players = self.players[self.players["Excluded"] == True].index

        rankings = rankings - rankings.loc[MatchRating.avg_player_name] + self.ranking_boost
        rankings = rankings.drop(excluded_players, errors="ignore")
        rankings = self.players.merge(rankings, left_index=True, right_index=True)
        rankings = rankings.drop(columns=["Excluded"])
        rankings = rankings.sort_values(by="Rating", ascending=False).reset_index()
        rankings["Rating"] = rankings["Rating"].astype(int)
        rankings.index += 1

        return rankings
    
    def __calculate_ratings(self, step_size):
        
        total_match_weights  = self.matches["K"].sum()

        for i in range(MatchRating.max_iterations):
            rating_table = self.__create_rating_table(step_size)
            
            match_score_accuracy = self.__calculate_match_score_accuracy(rating_table)        
            
            player_rating_accuracy = pow(match_score_accuracy.prod(), 1 / total_match_weights)
            accuracy_product = player_rating_accuracy.sort_values(ascending=False)

            best_accuracy = accuracy_product.index[0]
            print(f"Step Size={step_size}, Iteration {i + 1}, Accuracy: {round(accuracy_product.iloc[0], 5)}")

            if best_accuracy == MatchRating.original_rating_name:
                return self.__player_ratings.sort_values(by="Rating", ascending=False)
            
            player, rating_change = best_accuracy            
            self.__player_ratings.loc[player] += rating_change
            
        return self.__player_ratings.sort_values(by="Rating", ascending=False)

    def __calculate_match_score_accuracy(self, rating_table: pd.DataFrame):

        match_columns = self.matches.columns.to_list()
        winner_columns = [column for column in match_columns if column.startswith("Winner")]
        loser_columns = [column for column in match_columns if column.startswith("Loser")]

        def apply_match_score_accuracy(match):
            winners = match[winner_columns]
            winners = winners[winners.notna()]
            winner_rating = rating_table.loc[winners].sum()
            
            losers = match[loser_columns]
            losers = losers[losers.notna()]
            loser_rating = rating_table.loc[losers].sum()

            accuracy = pow(winner_rating / (winner_rating + loser_rating), match["K"])

            return accuracy

        return self.matches.apply(apply_match_score_accuracy, axis=1, result_type="expand")

    def __create_rating_table(self, step_size):
        
        ratings = []
        columns = []

        original_rating = self.__player_ratings["Rating"]

        def add_rating(rating, name):
            ratings.append(rating.apply(lambda x: pow(self.rating_scale_base, (x / self.rating_scale_divisor))))
            columns.append(name)

        def add_alternative_rating(player, rating_change):
            alternative_rating = original_rating.copy()
            alternative_rating.loc[player] += rating_change
                
            add_rating(alternative_rating, (player, rating_change))

        add_rating(original_rating, MatchRating.original_rating_name)

        for player in self.players.index:
            add_alternative_rating(player, step_size)
            add_alternative_rating(player, -step_size)
            
        rating_table = pd.concat(ratings, axis=1)
        rating_table.columns = columns

        return rating_table

In [4]:
rating_system = MatchRating(
    # data_sheet_file="Data/Test.xlsx",
    data_sheet_file="Data/UCH_Data.xlsx",
    rating_scale_base=10,
    rating_scale_divisor=400,
    rating_base=1000,
    tie_in_weight=0.5
)

rankings = rating_system.calculate_ratings(step_sizes=[1024,512,256,128,64,32,16,8,4,2,1,0.5])
rankings

Step Size=1024, Iteration 1, Accuracy: 0.50766
Step Size=1024, Iteration 2, Accuracy: 0.50766
Step Size=512, Iteration 1, Accuracy: 0.51233
Step Size=512, Iteration 2, Accuracy: 0.5142
Step Size=512, Iteration 3, Accuracy: 0.51608
Step Size=512, Iteration 4, Accuracy: 0.51797
Step Size=512, Iteration 5, Accuracy: 0.51986
Step Size=512, Iteration 6, Accuracy: 0.52176
Step Size=512, Iteration 7, Accuracy: 0.52176
Step Size=256, Iteration 1, Accuracy: 0.52884
Step Size=256, Iteration 2, Accuracy: 0.53636
Step Size=256, Iteration 3, Accuracy: 0.55031
Step Size=256, Iteration 4, Accuracy: 0.56702
Step Size=256, Iteration 5, Accuracy: 0.57085
Step Size=256, Iteration 6, Accuracy: 0.57426
Step Size=256, Iteration 7, Accuracy: 0.57756
Step Size=256, Iteration 8, Accuracy: 0.581
Step Size=256, Iteration 9, Accuracy: 0.58282
Step Size=256, Iteration 10, Accuracy: 0.58464
Step Size=256, Iteration 11, Accuracy: 0.58592
Step Size=256, Iteration 12, Accuracy: 0.58721
Step Size=256, Iteration 13, Acc

,ShortName,Name,Rating
1,JP,ja_paucia,1531
2,MRF,Morfin,1484
3,IEY,Gigi,1477
4,RAI,Raito,1434
5,TNR,TN'ramon,1313
6,MAD,MAD MAN,1241
7,EMB,Ember,1196
8,VB,Variety_Bear,1177
9,SZP,szop,1175
10,LEX,LexieUCH,1061
